In [1]:
import requests
import bs4
import pandas as pd

In [3]:
verbs = pd.read_csv('hebrew_verbs.csv', sep = ';')

In [4]:
error_msg='BANNED'

In [22]:
links = []

In [23]:
n = 0
for verb in verbs['Verb']:
    if str(verb) != 'nan':
        n += 1
        req = requests.get('https://www.pealim.com/search/?q='+verb)
        if req.status_code == 521:
            print(error_msg)
            print(n)
            break
        parser = bs4.BeautifulSoup(req.text, 'lxml')
        div = parser.find('div',attrs={'class' : 'verb-search-lemma'})
        if div:
            for a in div.children:
                link = a.get('href')
                if link:
                    links.append(link)
                else:
                    print('no link - ' + verb)
        else:
            print(verb)

להשתנה
להתאחדים
להצתייעץ


In [24]:
links = pd.DataFrame(links, columns=['Link'])
links = links.drop_duplicates(subset = ['Link'])
links.to_csv('links.csv')

In [35]:
links = pd.read_csv('links.csv', sep = ',')

In [36]:
cookies = {'showmeaning':'off','translit':'isr','hebstyle':'bp'}

In [37]:
res = []

In [38]:
for link in links['Link']:
    req = requests.get('https://www.pealim.com' + link, cookies=cookies)
    if req.status_code == 521:
        print(error_msg)
        print(link)
        break
    verb_description = []
    parser = bs4.BeautifulSoup(req.text, 'lxml')
    p = parser.find('b')
    verb_description.append('https://www.pealim.com' + link)
    verb_description.append(p.text)
    div = parser.find('div',attrs={'class' : 'lead'})
    verb_description.append(div.text)
    tds =  parser.findAll('td',attrs={'class' : 'conj-td'})
    for td in tds:
        div = td.find('span',attrs={'class' : 'menukad'})
        verb_description.append(div.text)
    verb_description.append(parser.find('span',attrs={'class' : 'menukad'}).find('a').text) # root
    res.append(verb_description)

In [39]:
res = pd.DataFrame(res)

In [40]:
res = res.sort_values(by = [1]) # sort by binyan
res.to_csv('res.csv',sep = ';')